# Lightgbm model test notebook

In [4]:
# !brew install cmake libomp
# !pip install lightgbm

Running `brew update --auto-update`...
==> Homebrew collects anonymous analytics.
Read the analytics documentation (and how to opt-out) here:
  https://docs.brew.sh/Analytics
No analytics have been recorded yet (nor will be during this `brew` run).

==> Homebrew is run entirely by unpaid volunteers. Please consider donating:
  https://github.com/Homebrew/brew#donations

==> Auto-updated Homebrew!
Updated 2 taps (homebrew/core and homebrew/cask).
==> New Formulae
actions-batch              jtbl                       prjtrellis
appwrite                   k8sgpt                     proto
asmfmt                     kin                        protoc-gen-js
autobrr                    kiota                      ratchet
bluez                      ktfmt                      rathole
bpftop                     kubeshark                  rattler-build
c-blosc2                   kubetui                    rawdog
c3c                        libnsbmp                   redict
c4core                   

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # data visualization
import seaborn as sns # statistical data visualization

import lightgbm as lgb

In [25]:
X_train_full_df = pd.read_csv("X_train_tSVD_3000.csv")
X_test_full_df = pd.read_csv("X_test_tSVD_3000.csv")

In [26]:
pd.set_option('display.max_columns', 500)
X_train_full_df.head(2)

,Title,description,authors,image,previewLink,publisher,infoLink,categories,index,reviews number,average rating,median rating,min review date_x,min review date_y,weighted rating,date,year,description_language,tokens,tSVD1,tSVD2,tSVD3,tSVD4,tSVD5,tSVD6,tSVD7,tSVD8,tSVD9,tSVD10,tSVD11,tSVD12,tSVD13,tSVD14,tSVD15,tSVD16,tSVD17,tSVD18,tSVD19,tSVD20,tSVD21,tSVD22,tSVD23,tSVD24,tSVD25,tSVD26,tSVD27,tSVD28,tSVD29,tSVD30,tSVD31,tSVD32,tSVD33,tSVD34,tSVD35,tSVD36,tSVD37,tSVD38,tSVD39,tSVD40,tSVD41,tSVD42,tSVD43,tSVD44,tSVD45,tSVD46,tSVD47,tSVD48,tSVD49,tSVD50,tSVD51,tSVD52,tSVD53,tSVD54,tSVD55,tSVD56,tSVD57,tSVD58,tSVD59,tSVD60,tSVD61,tSVD62,tSVD63,tSVD64,tSVD65,tSVD66,tSVD67,tSVD68,tSVD69,tSVD70,tSVD71,tSVD72,tSVD73,tSVD74,tSVD75,tSVD76,tSVD77,tSVD78,tSVD79,tSVD80,tSVD81,tSVD82,tSVD83,tSVD84,tSVD85,tSVD86,tSVD87,tSVD88,tSVD89,tSVD90,tSVD91,tSVD92,tSVD93,tSVD94,tSVD95,tSVD96,tSVD97,tSVD98,tSVD99,tSVD100,tSVD101,tSVD102,tSVD103,tSVD104,tSVD105,tSVD106,tSVD107,tSVD108,tSVD109,tSVD110,tSVD111,tSVD112,tSVD113,tSVD114,tSVD115,tSVD116,tSVD117,tSVD118,tSVD119,tSVD120,tSVD121,tSVD122,tSVD123,tSVD124,tSVD125,tSVD126,tSVD127,tSVD128,tSVD129,tSVD130,tSVD131,tSVD132,tSVD133,tSVD134,tSVD135,tSVD136,tSVD137,tSVD138,tSVD139,tSVD140,tSVD141,tSVD142,tSVD143,tSVD144,tSVD145,tSVD146,tSVD147,tSVD148,tSVD149,tSVD150,tSVD151,tSVD152,tSVD153,tSVD154,tSVD155,tSVD156,tSVD157,tSVD158,tSVD159,tSVD160,tSVD161,tSVD162,tSVD163,tSVD164,tSVD165,tSVD166,tSVD167,tSVD168,tSVD169,tSVD170,tSVD171,tSVD172,tSVD173,tSVD174,tSVD175,tSVD176,tSVD177,tSVD178,tSVD179,tSVD180,tSVD181,tSVD182,tSVD183,tSVD184,tSVD185,tSVD186,tSVD187,tSVD188,tSVD189,tSVD190,tSVD191,tSVD192,tSVD193,tSVD194,tSVD195,tSVD196,tSVD197,tSVD198,tSVD199,tSVD200,tSVD201,tSVD202,tSVD203,tSVD204,tSVD205,tSVD206,tSVD207,tSVD208,tSVD209,tSVD210,tSVD211,tSVD212,tSVD213,tSVD214,tSVD215,tSVD216,tSVD217,tSVD218,tSVD219,tSVD220,tSVD221,tSVD222,tSVD223,tSVD224,tSVD225,tSVD226,tSVD227,tSVD228,tSVD229,tSVD230,tSVD231,...,tSVD2751,tSVD2752,tSVD2753,tSVD2754,tSVD2755,tSVD2756,tSVD2757,tSVD2758,tSVD2759,tSVD2760,tSVD2761,tSVD2762,tSVD2763,tSVD2764,tSVD2765,tSVD2766,tSVD2767,tSVD2768,tSVD2769,tSVD2770,tSVD2771,tSVD2772,tSVD2773,tSVD2774,tSVD2775,tSVD2776,tSVD2777,tSVD2778,tSVD2779,tSVD2780,tSVD2781,tSVD2782,tSVD2783,tSVD2784,tSVD2785,tSVD2786,tSVD2787,tSVD2788,tSVD2789,tSVD2790,tSVD2791,tSVD2792,tSVD2793,tSVD2794,tSVD2795,tSVD2796,tSVD2797,tSVD2798,tSVD2799,tSVD2800,tSVD2801,tSVD2802,tSVD2803,tSVD2804,tSVD2805,tSVD2806,tSVD2807,tSVD2808,tSVD2809,tSVD2810,tSVD2811,tSVD2812,tSVD2813,tSVD2814,tSVD2815,tSVD2816,tSVD2817,tSVD2818,tSVD2819,tSVD2820,tSVD2821,tSVD2822,tSVD2823,tSVD2824,tSVD2825,tSVD2826,tSVD2827,tSVD2828,tSVD2829,tSVD2830,tSVD2831,tSVD2832,tSVD2833,tSVD2834,tSVD2835,tSVD2836,tSVD2837,tSVD2838,tSVD2839,tSVD2840,tSVD2841,tSVD2842,tSVD2843,tSVD2844,tSVD2845,tSVD2846,tSVD2847,tSVD2848,tSVD2849,tSVD2850,tSVD2851,tSVD2852,tSVD2853,tSVD2854,tSVD2855,tSVD2856,tSVD2857,tSVD2858,tSVD2859,tSVD2860,tSVD2861,tSVD2862,tSVD2863,tSVD2864,tSVD2865,tSVD2866,tSVD2867,tSVD2868,tSVD2869,tSVD2870,tSVD2871,tSVD2872,tSVD2873,tSVD2874,tSVD2875,tSVD2876,tSVD2877,tSVD2878,tSVD2879,tSVD2880,tSVD2881,tSVD2882,tSVD2883,tSVD2884,tSVD2885,tSVD2886,tSVD2887,tSVD2888,tSVD2889,tSVD2890,tSVD2891,tSVD2892,tSVD2893,tSVD2894,tSVD2895,tSVD2896,tSVD2897,tSVD2898,tSVD2899,tSVD2900,tSVD2901,tSVD2902,tSVD2903,tSVD2904,tSVD2905,tSVD2906,tSVD2907,tSVD2908,tSVD2909,tSVD2910,tSVD2911,tSVD2912,tSVD2913,tSVD2914,tSVD2915,tSVD2916,tSVD2917,tSVD2918,tSVD2919,tSVD2920,tSVD2921,tSVD2922,tSVD2923,tSVD2924,tSVD2925,tSVD2926,tSVD2927,tSVD2928,tSVD2929,tSVD2930,tSVD2931,tSVD2932,tSVD2933,tSVD2934,tSVD2935,tSVD2936,tSVD2937,tSVD2938,tSVD2939,tSVD2940,tSVD2941,tSVD2942,tSVD2943,tSVD2944,tSVD2945,tSVD2946,tSVD2947,tSVD2948,tSVD2949,tSVD2950,tSVD2951,tSVD2952,tSVD2953,tSVD2954,tSVD2955,tSVD2956,tSVD2957,tSVD2958,tSVD2959,tSVD2960,tSVD2961,tSVD2962,tSVD2963,tSVD2964,tSVD2965,tSVD2966,tSVD2967,tSVD2968,tSVD2969,tSVD2970,tSVD2971,tSVD2972,tSVD2973,tSVD2974,tSVD2975,tSVD2976,tSVD297

## In this test the target variable will be "weighted rating"

### Definining X_train, X_test, y_train, y_test

In [33]:
columns_to_drop = ["Title","description","authors","image",
                   "previewLink","publisher","infoLink","categories",
                   "index","reviews number","average rating",
                   "median rating","min review date_x","min review date_y",
                   "weighted rating","date","description_language","tokens"]

X_train = X_train_full_df.drop(columns_to_drop, axis = 1)
X_test = X_test_full_df.drop(columns_to_drop, axis = 1)

# change the year feature to an integer
X_train["year"] = X_train["year"].astype(int)
X_test["year"] = X_test["year"].astype(int)

y_train = X_train_full_df["weighted rating"]
y_test = X_test_full_df["weighted rating"]

### Hyperparameters for lightgbm

In [37]:
hyper_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['l1','l2'],
    'learning_rate': 0.005,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.7,
    'bagging_freq': 10,
    'verbose': 0,
    "max_depth": 8,
    "num_leaves": 128,  
    "max_bin": 512,
    "num_iterations": 100000
}

In [ ]:
# Instantiating and fitting the lightgbm model
gbm = lgb.LGBMRegressor(**hyper_params)

gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='l1',
        callbacks=[
        lgb.early_stopping(stopping_rounds=3),
    ])

/Users/meganguidry/miniconda3/lib/python3.11/site-packages/lightgbm/engine.py:172: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.7, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7


In [ ]:
y_pred = gbm.predict(X_train, num_iteration=bgm.best_iteration_)

In [ ]:
# Basic RMSE
print('The rmse of prediction is:', round(mean_squared_log_error(y_pred, y_train) ** 0.5, 5))